# RandomVector

## Résumé

Dans ce document, nous présentons la classe `RandomVector`. Nous présentons en particulier le lien et les différences avec la classe `JointDistribution`. 

[Pour voir ce Jupyter Notebook, nous conseillons NBViewer.](https://nbviewer.org/github/mbaudin47/otsupgalilee-eleve/blob/master/1-Intro-OT/RandomVector.ipynb)

## Références

- http://openturns.github.io/openturns/master/user_manual/_generated/openturns.RandomVector.html
- http://openturns.github.io/openturns/master/user_manual/_generated/openturns.JointDistribution.html
- http://openturns.github.io/openturns/master/user_manual/_generated/openturns.CompositeDistribution.html
- http://openturns.github.io/openturns/master/examples/probabilistic_modeling/composite_distribution.html

## Introduction

OpenTURNS fournit deux classes « associées » aux fonctions :

- `JointDistribution` : une distribution multidimensionnelle définie par des marginales et une copule ;
- `RandomVector` : un vecteur aléatoire.

On peut créer un `RandomVector` de deux manières :

- à partir d’une `Distribution` ;
- l’application d’une fonction **g** à un `RandomVector`.

*Objectifs* :

- comprendre le lien entre un `RandomVector` et une fonction ;
- comprendre le rôle de la classe `RandomVector`.

In [1]:
import openturns as ot

Une `JointDistribution` est composée :

- des marginales ;
- une copule (par défaut, c’est la copule indépendante).

Exemple : on considère trois variables gaussiennes indépendantes et on souhaite créer la distribution associée. 

In [2]:
inputDistribution = ot.Normal(3)

Pour créer un `RandomVector` à partir d’une distribution on utilise le constructeur suivant :

In [3]:
inputRandomVector = ot.RandomVector(inputDistribution)

Maintenant, on va pouvoir lui appliquer une fonction **g**.


Pour créer un `RandomVector` à partir d’une fonction **g**, il faut :

- un `RandomVector` en entrée ;
- une `Function` à appliquer sur l’entrée.

Dans l'exemple suivant, on crée un `RandomVector` à partir d’un `RandomVector` et d’une fonction **g**. On commence par créer une fonction avec l'opérateur Python `def`. 

In [4]:
def simulator(x):
    y0 = x[0] + x[1] + x[2]
    y1 = x[0] - x[1] * x[2]
    y = [y0, y1]
    return y

Puis on utilise les classes `PythonFunction` et `RandomVector` pour créer le vecteur de sortie.

In [5]:
function = ot.PythonFunction(3, 2, simulator)
outputRandomVector = ot.CompositeRandomVector(function, inputRandomVector)

## Points communs et différences entre `RandomVector` et `JointDistribution`

Points communs entre `RandomVector` et `JointDistribution` :

- la méthode `getSample` est commune aux deux classes.

Différences entre `RandomVector` et `JointDistribution` :

- un `RandomVector` n’a pas de méthode pour évaluer sa PDF : la méthode `RandomVector.computePDF()` n'existe pas. En général, il faudrait avoir toute la distribution de $Y = g(\boldsymbol{X})$ pour l’avoir. En général, lorsque **g** est un code de calcul externe de type boîte noire, c’est très coûteux, voire impossible.
- une `JointDistribution` dispose des méthodes pour évaluer la PDF, la CDF, les quantiles, etc. En d’autres termes, toute la distribution est connue. Exemple : la méthode `computeCDF()`.

**Remarque.** Pour améliorer les performances, nous pourrions implémenter une version vectorisée utilisant NumPy.

## Deux cas particuliers de `RandomVector`

- Cas 1 : Le `RandomVector` d’entrée a été construit à partir d’une
`JointDistribution` qu’on peut récupérer grâce à la méthode
`getDistribution`.


In [6]:
inputRandomVector.getDistribution()

class=Normal name=Normal dimension=3 mean=class=Point name=Unnamed dimension=3 values=[0,0,0] sigma=class=Point name=Unnamed dimension=3 values=[1,1,1] correlationMatrix=class=CorrelationMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[1,0,0,0,1,0,0,0,1]

- Cas 2 : Le `RandomVector` de sortie a été construit à partir d'une fonction : la méthode `getDistribution` échoue, et c’est normal.

In [7]:
# Génère une exception
# outputRandomVector.getDistribution()

## Exercices

### Exercice 1 : une fonction à trois entrées

Définir la fonction `symbolicSimulator` comme une `SymbolicFunction` implémentant la fonction suivante :
$$
\begin{aligned}
Y_1 &= X_1 + X_2 + X_3, \\
Y_2 &= X_1 - X_2 X_3.
\end{aligned}
$$

On suppose que $X_i \sim \mathcal{N}(0, 1)$ pour $i \in \{1, 2, 3\}$ et que les variables $X_1$, $X_2$ et $X_3$ sont indépendantes.

Comment créer le `RandomVector` associé au vecteur $\boldsymbol{Y}=(Y_1, Y_2)^T$?

### Exercice 2 : quatre conversions

Expérimenter les quatre conversions présentées ci-dessous :

1. `RandomVector` → `RandomVector`
1. `Distribution` → `Distribution`
1. `Distribution` → `RandomVector`
1. `RandomVector` → `Distribution`

**Questions**

- Quelles sont les conversions possibles ?
- Pourquoi certaines conversions sont elles impossibles ?

### Exercice 3 : composition de RandomVector

On considère la fonction `simulator2`.

In [8]:
def simulator2(x):
    y0 = x[0] + x[1]
    y1 = x[1] ** 2
    y = [y0, y1]
    return y

**Questions**

- Utiliser la classe `RandomVector` pour définir le vecteur aléatoire associé à la composition de la fonction `simulator` par la fonction `simulator2`, c’est-à-dire `Y = simulator2(simulator(X))`.
- Utiliser la classe `ComposedFunction` pour définir la fonction `simulator3` créée par composition de `simulator2` et `simulator`. Puis créer le `RandomVector` associé à `simulator3`. Ici, on crée une fonction `simulator3(x) = simulator2(simulator(x)` puis on crée le `RandomVector` défini par `simulator3` et `inputRandomVector`. 

## Exercice 4 :  la classe CompositeDistribution

La classe `CompositeDistribution` permet de définir une distribution fondée sur l'application d'une fonction scalaire (de $\mathbb{R}$ dans $\mathbb{R}$) à une distribution unidimensionnelle. 
Considérons **X** une variable aléatoire de loi $\mathcal{L}_X$ et $g : \mathbb{R} \rightarrow \mathbb{R}$ une fonction. On considère la variable aléatoire **Y** définie par :
$$
Y = g(X).
$$
On note $\mathcal{L}_Y$ la distribution de la variable **Y** : la classe `CompositeDistribution` permet de définir la loi de **Y**. 

On considère la variable **X** de loi uniforme entre 0 et 12. On considère la fonction **g** définie par :
$$
g(x) = x^2
$$
pour $x \in [0,12]$.

**Questions**

- Définir la variable `distributionX` associée à la loi de la variable **X**. Dessiner la densité de probabilité de la distribution.
- Définir la fonction `maFonc` associée à la fonction **g**. Dessiner la fonction entre 0 et 12.
- Utiliser la classe `CompositeDistribution` pour définir la distribution associée à **Y**. Dessiner cette distribution.